# Day 13: Intrinsic Value & DCF Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class3_trading_strategies/week3_value_growth/day13_intrinsic_value_dcf.ipynb)

---

## Learning Objectives

1. **Understand** the concept of intrinsic value
2. **Learn** Discounted Cash Flow (DCF) methodology
3. **Calculate** present value of future cash flows
4. **Apply** multiple valuation methods
5. **Build** a simple DCF model in Python

---

## Lecture (30 minutes)

### What is Intrinsic Value?

**Intrinsic value** is the calculated worth of a company based on fundamental analysis, independent of its current market price.

```
INTRINSIC VALUE CONCEPT
=======================

"The intrinsic value of a company is the
present value of all future cash flows."
                        - Warren Buffett

KEY INSIGHT:
- A dollar today is worth more than a dollar tomorrow
- Future cash flows must be discounted to present value
- Sum of all discounted cash flows = intrinsic value

VALUATION FRAMEWORK:

Year 1    Year 2    Year 3    Year N    Terminal
  CF1      CF2       CF3       CFn       Value
   |        |         |         |          |
   v        v         v         v          v
 [Discount all cash flows to present value]
   |        |         |         |          |
   +--------+---------+---------+----------+
                      |
                      v
              INTRINSIC VALUE
```

### Time Value of Money

```
TIME VALUE OF MONEY
===================

PRESENT VALUE FORMULA:
PV = FV / (1 + r)^n

Where:
- PV = Present Value
- FV = Future Value
- r = Discount rate (required return)
- n = Number of periods

EXAMPLE:
$100 in 5 years at 10% discount rate:
PV = $100 / (1.10)^5
PV = $100 / 1.61
PV = $62.09

IMPACT OF DISCOUNT RATE:
$100 in 10 years:
- At 5%:  $61.39
- At 10%: $38.55
- At 15%: $24.72

Higher discount rate = lower present value
```

### DCF Model Overview

```
DCF MODEL STRUCTURE
===================

STEP 1: PROJECT FREE CASH FLOWS
- Forecast revenue growth (5-10 years)
- Estimate profit margins
- Calculate free cash flow each year

FREE CASH FLOW (FCF) = 
  Operating Cash Flow - Capital Expenditures

Or simplified:
FCF = EBIT × (1-tax) + Depreciation - CapEx - ΔNWC

STEP 2: DETERMINE DISCOUNT RATE (WACC)
WACC = (E/V × Re) + (D/V × Rd × (1-T))

Where:
- E = Market value of equity
- D = Market value of debt
- V = E + D
- Re = Cost of equity (CAPM)
- Rd = Cost of debt
- T = Tax rate

Simplified: Use 8-12% for most stocks

STEP 3: CALCULATE TERMINAL VALUE
TV = FCFn × (1 + g) / (r - g)

Where:
- FCFn = Final year free cash flow
- g = Perpetual growth rate (2-3%)
- r = Discount rate

STEP 4: DISCOUNT ALL CASH FLOWS
Intrinsic Value = Σ FCFt/(1+r)^t + TV/(1+r)^n

STEP 5: CALCULATE PER SHARE VALUE
Value Per Share = (Intrinsic Value - Debt + Cash) / Shares
```

### Simplified DCF Methods

```
SIMPLIFIED VALUATION METHODS
============================

1. EARNINGS POWER VALUE (EPV)
   EPV = Normalized Earnings / Cost of Capital
   
   Example:
   Earnings = $5/share
   Cost of Capital = 10%
   EPV = $5 / 0.10 = $50/share

2. GORDON GROWTH MODEL (DDM)
   Value = D1 / (r - g)
   
   Where:
   - D1 = Next year's dividend
   - r = Required return
   - g = Dividend growth rate
   
   Example:
   Dividend = $2, Growth = 5%, Required = 10%
   Value = $2 / (0.10 - 0.05) = $40

3. REVERSE DCF
   - Input current price
   - Calculate implied growth rate
   - Ask: Is this realistic?

4. TWO-STAGE MODEL
   - High growth phase (5-10 years)
   - Stable growth phase (perpetuity)
   - More realistic for growth companies
```

### Key DCF Assumptions

```
CRITICAL ASSUMPTIONS
====================

DISCOUNT RATE:
- Risk-free rate: 10Y Treasury (~4-5%)
- Equity risk premium: 4-6%
- Company-specific risk: 0-5%
- Typical range: 8-15%

GROWTH RATES:
- High growth phase: Company specific
- Terminal growth: GDP growth (~2-3%)
- Never exceed discount rate!

MARGIN ASSUMPTIONS:
- Historical average as baseline
- Industry benchmarks
- Competitive dynamics

SENSITIVITY:
Small changes in assumptions = big value changes
Always run sensitivity analysis!

DCF WARNINGS:
- Garbage in = Garbage out
- More precision ≠ more accuracy
- Use ranges, not point estimates
- Cross-check with other methods
```

---

## Hands-On Practice (15 minutes)

In [ ]:
!pip install yfinance pandas numpy matplotlib -q
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print("Libraries loaded!")

In [ ]:
def present_value(future_value, rate, periods):
    """Calculate present value of a future amount."""
    return future_value / (1 + rate) ** periods

def demonstrate_time_value():
    """Demonstrate time value of money."""
    print(f"\n{'='*50}")
    print("TIME VALUE OF MONEY DEMONSTRATION")
    print(f"{'='*50}\n")
    
    future_amount = 1000
    rates = [0.05, 0.10, 0.15]
    years = range(1, 11)
    
    print(f"Future Value: ${future_amount:,}")
    print(f"\nPresent Value at Different Discount Rates:")
    print(f"{'-'*50}")
    print(f"{'Year':<8}{'5%':>12}{'10%':>12}{'15%':>12}")
    print(f"{'-'*50}")
    
    for year in years:
        pvs = [present_value(future_amount, r, year) for r in rates]
        print(f"{year:<8}${pvs[0]:>10,.2f}${pvs[1]:>10,.2f}${pvs[2]:>10,.2f}")

demonstrate_time_value()

In [ ]:
def simple_dcf(ticker, growth_rate=0.10, terminal_growth=0.03, 
               discount_rate=0.10, projection_years=5):
    """Simple DCF valuation model."""
    stock = yf.Ticker(ticker)
    info = stock.info
    cashflow = stock.cashflow
    
    # Get current free cash flow
    try:
        operating_cf = cashflow.loc['Operating Cash Flow'].iloc[0]
        capex = abs(cashflow.loc['Capital Expenditure'].iloc[0]) if 'Capital Expenditure' in cashflow.index else 0
        fcf = operating_cf - capex
    except:
        fcf = info.get('freeCashflow', 0)
    
    shares = info.get('sharesOutstanding', 1)
    current_price = info.get('currentPrice', info.get('regularMarketPrice', 0))
    cash = info.get('totalCash', 0)
    debt = info.get('totalDebt', 0)
    
    print(f"\n{'='*60}")
    print(f"DCF VALUATION: {info.get('shortName', ticker)}")
    print(f"{'='*60}\n")
    
    print(f"Current Free Cash Flow: ${fcf/1e9:.2f}B")
    print(f"Shares Outstanding: {shares/1e9:.2f}B")
    print(f"Current Price: ${current_price:.2f}")
    
    # Project future cash flows
    print(f"\n{'-'*60}")
    print(f"PROJECTED FREE CASH FLOWS (Growth: {growth_rate*100:.0f}%)")
    print(f"{'-'*60}")
    
    projected_fcf = []
    present_values = []
    
    for year in range(1, projection_years + 1):
        fcf_year = fcf * (1 + growth_rate) ** year
        pv = present_value(fcf_year, discount_rate, year)
        projected_fcf.append(fcf_year)
        present_values.append(pv)
        print(f"Year {year}: FCF = ${fcf_year/1e9:.2f}B | PV = ${pv/1e9:.2f}B")
    
    # Terminal value
    terminal_fcf = projected_fcf[-1] * (1 + terminal_growth)
    terminal_value = terminal_fcf / (discount_rate - terminal_growth)
    terminal_pv = present_value(terminal_value, discount_rate, projection_years)
    
    print(f"\nTerminal Value: ${terminal_value/1e9:.2f}B")
    print(f"Terminal PV: ${terminal_pv/1e9:.2f}B")
    
    # Total intrinsic value
    total_pv_fcf = sum(present_values)
    enterprise_value = total_pv_fcf + terminal_pv
    equity_value = enterprise_value + cash - debt
    intrinsic_per_share = equity_value / shares
    
    print(f"\n{'-'*60}")
    print("VALUATION SUMMARY")
    print(f"{'-'*60}")
    print(f"Sum of PV(FCF): ${total_pv_fcf/1e9:.2f}B")
    print(f"+ PV(Terminal): ${terminal_pv/1e9:.2f}B")
    print(f"= Enterprise Value: ${enterprise_value/1e9:.2f}B")
    print(f"+ Cash: ${cash/1e9:.2f}B")
    print(f"- Debt: ${debt/1e9:.2f}B")
    print(f"= Equity Value: ${equity_value/1e9:.2f}B")
    print(f"\nIntrinsic Value Per Share: ${intrinsic_per_share:.2f}")
    print(f"Current Market Price: ${current_price:.2f}")
    
    # Margin of safety
    margin = (intrinsic_per_share - current_price) / intrinsic_per_share * 100
    
    print(f"\n{'-'*60}")
    print("VERDICT")
    print(f"{'-'*60}")
    print(f"Margin of Safety: {margin:.1f}%")
    
    if margin > 25:
        print("\nUNDERVALUED - Significant margin of safety")
    elif margin > 0:
        print("\nFAIRLY VALUED - Trading below intrinsic value")
    else:
        print("\nOVERVALUED - Trading above intrinsic value")
    
    return {
        'intrinsic_value': intrinsic_per_share,
        'current_price': current_price,
        'margin_of_safety': margin
    }

# Run DCF on Apple
result = simple_dcf('AAPL', growth_rate=0.08, discount_rate=0.10)

In [ ]:
def sensitivity_analysis(ticker, base_growth=0.10, base_discount=0.10):
    """Run sensitivity analysis on DCF assumptions."""
    stock = yf.Ticker(ticker)
    info = stock.info
    cashflow = stock.cashflow
    
    # Get FCF
    try:
        operating_cf = cashflow.loc['Operating Cash Flow'].iloc[0]
        capex = abs(cashflow.loc['Capital Expenditure'].iloc[0]) if 'Capital Expenditure' in cashflow.index else 0
        fcf = operating_cf - capex
    except:
        fcf = info.get('freeCashflow', 0)
    
    shares = info.get('sharesOutstanding', 1)
    cash = info.get('totalCash', 0)
    debt = info.get('totalDebt', 0)
    current_price = info.get('currentPrice', info.get('regularMarketPrice', 0))
    
    # Growth and discount rate ranges
    growth_rates = [0.05, 0.08, 0.10, 0.12, 0.15]
    discount_rates = [0.08, 0.10, 0.12]
    terminal_growth = 0.03
    years = 5
    
    print(f"\n{'='*70}")
    print(f"SENSITIVITY ANALYSIS: {info.get('shortName', ticker)}")
    print(f"{'='*70}")
    print(f"\nCurrent Price: ${current_price:.2f}\n")
    
    # Create sensitivity table
    results = []
    
    for dr in discount_rates:
        row = {'Discount Rate': f"{dr*100:.0f}%"}
        for gr in growth_rates:
            # Quick DCF calculation
            pv_fcf = sum([present_value(fcf * (1+gr)**y, dr, y) for y in range(1, years+1)])
            term_fcf = fcf * (1+gr)**years * (1+terminal_growth)
            term_value = term_fcf / (dr - terminal_growth)
            term_pv = present_value(term_value, dr, years)
            ev = pv_fcf + term_pv
            equity = ev + cash - debt
            per_share = equity / shares
            row[f"{gr*100:.0f}%"] = per_share
        results.append(row)
    
    df = pd.DataFrame(results)
    df = df.set_index('Discount Rate')
    
    print("INTRINSIC VALUE PER SHARE")
    print("(Rows: Discount Rate, Columns: Growth Rate)")
    print(f"{'-'*70}")
    print(df.round(2).to_string())
    
    # Visual heatmap
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Convert to numpy for heatmap
    values = df.values
    
    # Color based on margin of safety
    colors = np.where(values > current_price * 1.25, 'green',
                     np.where(values > current_price, 'lightgreen', 'salmon'))
    
    im = ax.imshow(values > current_price, cmap='RdYlGn', aspect='auto')
    
    # Labels
    ax.set_xticks(range(len(growth_rates)))
    ax.set_xticklabels([f"{r*100:.0f}%" for r in growth_rates])
    ax.set_yticks(range(len(discount_rates)))
    ax.set_yticklabels([f"{r*100:.0f}%" for r in discount_rates])
    
    # Add text annotations
    for i in range(len(discount_rates)):
        for j in range(len(growth_rates)):
            text = ax.text(j, i, f"${values[i, j]:.0f}",
                          ha="center", va="center", fontsize=10)
    
    ax.set_xlabel('Growth Rate')
    ax.set_ylabel('Discount Rate')
    ax.set_title(f'{ticker} DCF Sensitivity\n(Green = Above Current Price ${current_price:.0f})')
    
    plt.tight_layout()
    plt.show()
    
    return df

sensitivity_df = sensitivity_analysis('AAPL')

In [ ]:
def dividend_discount_model(ticker):
    """Value a stock using Gordon Growth Model (DDM)."""
    stock = yf.Ticker(ticker)
    info = stock.info
    
    dividend = info.get('dividendRate', 0)
    price = info.get('currentPrice', info.get('regularMarketPrice', 0))
    
    if dividend == 0:
        print(f"{ticker} does not pay a dividend. DDM not applicable.")
        return None
    
    print(f"\n{'='*50}")
    print(f"DIVIDEND DISCOUNT MODEL: {ticker}")
    print(f"{'='*50}\n")
    
    print(f"Current Annual Dividend: ${dividend:.2f}")
    print(f"Current Price: ${price:.2f}")
    print(f"Current Yield: {(dividend/price)*100:.2f}%")
    
    # Different growth assumptions
    growth_rates = [0.02, 0.03, 0.04, 0.05]
    required_return = 0.10
    
    print(f"\n{'-'*50}")
    print(f"Gordon Growth Model (Required Return: {required_return*100:.0f}%)")
    print(f"{'-'*50}")
    
    for g in growth_rates:
        if g < required_return:
            d1 = dividend * (1 + g)  # Next year's dividend
            value = d1 / (required_return - g)
            margin = (value - price) / value * 100
            print(f"Growth {g*100:.0f}%: Fair Value = ${value:.2f} (MoS: {margin:.1f}%)")
        else:
            print(f"Growth {g*100:.0f}%: Invalid (g >= r)")
    
    return dividend

# Test on a dividend stock
dividend_discount_model('JNJ')

In [ ]:
def multi_method_valuation(ticker):
    """Compare multiple valuation methods."""
    stock = yf.Ticker(ticker)
    info = stock.info
    
    price = info.get('currentPrice', info.get('regularMarketPrice', 0))
    eps = info.get('trailingEps', 0)
    book_value = info.get('bookValue', 0)
    fcf_per_share = info.get('freeCashflow', 0) / info.get('sharesOutstanding', 1) if info.get('sharesOutstanding') else 0
    dividend = info.get('dividendRate', 0)
    
    print(f"\n{'='*60}")
    print(f"MULTI-METHOD VALUATION: {info.get('shortName', ticker)}")
    print(f"{'='*60}")
    print(f"\nCurrent Price: ${price:.2f}\n")
    
    valuations = []
    
    # 1. Graham Number
    if eps > 0 and book_value > 0:
        graham = np.sqrt(22.5 * eps * book_value)
        valuations.append({'Method': 'Graham Number', 'Value': graham})
    
    # 2. Earnings Power Value
    if eps > 0:
        epv = eps / 0.10  # 10% cost of capital
        valuations.append({'Method': 'Earnings Power Value', 'Value': epv})
    
    # 3. P/E based (industry average ~20)
    if eps > 0:
        pe_value = eps * 15  # Conservative P/E of 15
        valuations.append({'Method': 'P/E (15x)', 'Value': pe_value})
    
    # 4. Book Value
    if book_value > 0:
        valuations.append({'Method': 'Book Value', 'Value': book_value})
    
    # 5. DDM (if dividend paying)
    if dividend > 0:
        ddm_value = (dividend * 1.03) / (0.10 - 0.03)  # 3% growth, 10% return
        valuations.append({'Method': 'DDM (3% growth)', 'Value': ddm_value})
    
    # 6. FCF Yield based
    if fcf_per_share > 0:
        fcf_value = fcf_per_share / 0.05  # 5% FCF yield
        valuations.append({'Method': 'FCF (5% yield)', 'Value': fcf_value})
    
    # Display results
    df = pd.DataFrame(valuations)
    df['vs Price'] = ((df['Value'] - price) / price * 100).round(1)
    df['Rating'] = df['vs Price'].apply(lambda x: 'Undervalued' if x > 20 else ('Overvalued' if x < -20 else 'Fair'))
    
    print(df.to_string(index=False))
    
    # Average
    avg_value = df['Value'].mean()
    avg_diff = (avg_value - price) / price * 100
    
    print(f"\n{'-'*60}")
    print(f"Average Intrinsic Value: ${avg_value:.2f}")
    print(f"vs Current Price: {avg_diff:+.1f}%")
    
    return df

multi_method_valuation('AAPL')

---

## Quiz

In [ ]:
quiz = [
    {"q": "What is intrinsic value based on?", "a": "B", 
     "opts": ["A) Current market price", "B) Present value of future cash flows", 
              "C) Historical average price", "D) Analyst recommendations"]},
    {"q": "A higher discount rate results in?", "a": "A",
     "opts": ["A) Lower present value", "B) Higher present value", 
              "C) No change in value", "D) More volatility"]},
    {"q": "What is terminal value?", "a": "C",
     "opts": ["A) Value at company founding", "B) Liquidation value", 
              "C) Value of all cash flows beyond projection period", "D) Book value"]},
    {"q": "In DCF, what growth rate should terminal growth never exceed?", "a": "B",
     "opts": ["A) Historical growth", "B) The discount rate", 
              "C) Industry average", "D) 10%"]},
    {"q": "The Gordon Growth Model is best for?", "a": "D",
     "opts": ["A) High-growth tech stocks", "B) Unprofitable companies", 
              "C) Cyclical companies", "D) Mature dividend-paying stocks"]}
]

def run_quiz():
    for i, q in enumerate(quiz):
        print(f"\nQ{i+1}: {q['q']}")
        for opt in q['opts']:
            print(f"   {opt}")
    print("\n" + "="*50)
    print("ANSWERS: 1-B, 2-A, 3-C, 4-B, 5-D")
    
run_quiz()

---

## Summary

- **Intrinsic value** = Present value of all future cash flows
- **DCF steps**: Project FCF → Discount → Add terminal value → Subtract debt
- **Key assumptions**: Growth rate, discount rate, terminal growth
- **Sensitivity analysis** is critical - small changes have big impacts
- **Use multiple methods** and cross-check valuations

**Tomorrow**: Day 14 - Growth Investing